In [1]:
ORSkey='5b3ce3597851110001cf6248b738e00efacb4d2db3104c633c184900'

In [59]:
import numpy as np
import pandas as pd
import folium
import ast
import openrouteservice as ors
import random

In [36]:
locations = pd.read_csv("FoodstuffsLocations.csv")
locations.head()

,Type,Location,Supermarket,Long,Lat
0,New World,Albany,New World Albany,174.710513,-36.728144
1,New World,Birkenhead,New World Birkenhead,174.711486,-36.811428
2,New World,Botany,New World Botany,174.911491,-36.933883
3,New World,Browns Bay,New World Browns Bay,174.747287,-36.715961
4,New World,Devonport,New World Devonport,174.796193,-36.829510


In [38]:
selected_saturday = pd.read_csv("selected_saturday.csv")
selected_saturday.head()

,Route,Truck,Cost
0,"['Warehouse', 'New World Mt Roskill', ""Pak 'n ...",Normal,109.761667
1,"['Warehouse', 'Four Square Fair Price Henderso...",Normal,223.661667
2,"['Warehouse', 'New World Botany', ""Pak 'n Save...",Normal,270.636250
3,"['Warehouse', ""Pak 'n Save Sylvia Park"", 'New ...",Normal,186.352500
4,"['Warehouse', ""Pak 'n Save Glen Innes"", 'Four ...",Normal,227.462500


In [39]:
selected_weekday = pd.read_csv("selected_weekday.csv")
selected_weekday.head()

,Route,Truck,Cost
0,"['Warehouse', ""Pak 'n Save Botany"", 'New World...",Normal,205.303750
1,"['Warehouse', ""Pak 'n Save Manukau"", 'New Worl...",Normal,146.885417
2,"['Warehouse', 'New World Eastridge', 'New Worl...",Normal,159.403750
3,"['Warehouse', 'New World Ormiston', ""Pak 'n Sa...",Normal,197.378333
4,"['Warehouse', ""Pak 'n Save Royal Oak"", 'Wareho...",Normal,60.860833


In [25]:
def initialise_map(locations):
    coords = locations[['Long', 'Lat']] # Mapping packages work with Long, Lat arrays
    coords = coords.to_numpy().tolist() # Make the arrays into a list of lists.
    # Folium, however, requires Lat, Long arrays - so a reversal is needed.
    # coords[0] is the warehouse
    m = folium.Map(location = list(reversed(coords[2])), zoom_start=10.25)
    for i in range(len(locations)):
        if locations.Type[i] == "Four Square":
            iconCol = "green"
        elif locations.Type[i] == "New World":
            iconCol = "red"
        elif locations.Type[i] == "Pak 'n Save":
            iconCol = "orange"
        elif locations.Type[i] == "Warehouse":
            iconCol = "black"
        folium.Marker(list(reversed(coords[i])), popup = locations.Supermarket[i], icon = folium.Icon(color = iconCol)).add_to(m)
    return m

In [56]:
def find_driving_route(coords):
    client = ors.Client(key=ORSkey)
    
    return client.directions(
        coordinates = coords,
        profile = 'driving-hgv', # can be driving-car, driving-hgv, etc.
        format='geojson',
        validate = False
    )

In [60]:
def plot_routes(locations, routes, m):
    route_list = [ast.literal_eval(route) for route in routes.Route.values.tolist()]
    for route in route_list:
        coord_list = []
        for node in route:
            coord = locations[locations.Supermarket==node][['Long', 'Lat']]
            coord = coord.to_numpy().tolist()
            coord_list.append(coord[0])
        
        driving_route = find_driving_route(coord_list)
        coordinates = driving_route['features'][0]['geometry']['coordinates']
        folium.PolyLine(
            locations = [list(reversed(coord)) for coord in coordinates],
            color = '#' + ''.join([random.choice('ABCDEF0123456789') for i in range(6)])
        ).add_to(m)
    return m

In [28]:
initialise_map(locations)

In [61]:
saturday_map = initialise_map(locations)
plot_routes(locations, selected_saturday, saturday_map)

In [62]:
weekday_map = initialise_map(locations)
plot_routes(locations, selected_weekday, weekday_map)